In [ ]:
# !pip install requests
# !pip install bs4
# !pip install selenium
# !pip install numpy

In [7]:
from urllib.request import Request
from urllib.request import urlopen
from bs4 import BeautifulSoup
import urllib.request
import re
import numpy as np
import time
import os
import random
import time
import pandas as pd

#  투순지 타잔지에 따라 달라짐, https://statiz.sporki.com/player/?m=day&p_no=14788&pos=pitching&year=2021



def player_score_batting(year, p_no):
    url = f'https://statiz.sporki.com/player/?m=year&m2=batting&m3=default&p_no={p_no}'
    req = Request(url,headers={'User-Agent':'Mozila/5.0'})
    webpage = urlopen(req)
    soup = BeautifulSoup(webpage,'html.parser')

    element = soup.find_all(class_ = 'table_type02 transverse_scroll')[0]
    headers = [th['tooltip'] for th in element.find_all('th')]
    headers = headers[:32] # 32개만 header로 존재,
    headers.remove('Ratio, 비율')

    data = []
    for row in element.find_all('tr'):
        row_data = [td.text for td in row.find_all('td')]
        if row_data:
            data.append(row_data)

    # 데이터프레임 생성
    df = pd.DataFrame(data[:2], columns=headers)

    return df


def player_score_pitching(year, p_no):
    url = f'https://statiz.sporki.com/player/?m=year&m2=pitching&m3=default&p_no={p_no}'
    req = Request(url,headers={'User-Agent':'Mozila/5.0'})
    webpage = urlopen(req)
    soup = BeautifulSoup(webpage,'html.parser')

    element = soup.find_all(class_ = 'table_type02 transverse_scroll')[0]
    headers = [th['tooltip'] for th in element.find_all('th')]
    headers = headers[:34] # 32개만 header로 존재,

    data = []
    for row in element.find_all('tr'):
        row_data = [td.text for td in row.find_all('td')]
        if row_data:
            data.append(row_data)

    # 데이터프레임 생성
    df = pd.DataFrame(data[:2], columns=headers)

    return df

In [8]:
player = {'이의리':{'p_no':'14788',
                    'pos':'pitching',
                    'year':[2021,2022]},
          '이정후':{'p_no':'12906',
                    'pos':'batting',
                    'year':[2017,2018]},
         '정은원':{'p_no':'13137',
                    'pos':'batting',
                    'year':[2018,2019]}}


In [9]:
batting = pd.DataFrame()
pitching = pd.DataFrame()
for p_name in list(player.keys()):
    pos = player[p_name]['pos']
    if pos == 'batting':
        no = player[p_name]['p_no']
        year = player[p_name]['year']
        player_df = player_score_batting(year, no)
        player_df['name'] = p_name
        player_df['p_no'] = no
        player_df['p_pos'] = pos

        batting = pd.concat([batting, player_df])

    else:
        no = player[p_name]['p_no']
        year = player[p_name]['year']
        player_df = player_score_pitching(year, no)
        player_df['name'] = p_name
        player_df['p_no'] = no
        player_df['p_pos'] = pos

        pitching = pd.concat([pitching, player_df])


In [ ]:
batting.columns

Index(['Year, 연도', 'Team, 팀', 'Age, 나이(만)', 'Position, 포지션', 'Games, 출장',
       'Plate Appearance, 타석', 'effective Plate Appearance, 실질타석(유효타석)',
       'At Bat, 타수', 'Runs , 득점', 'Hits, 안타', 'Double, 2루타', 'Triple, 3루타',
       'Home Run, 홈런', 'Total Bases, 루타', 'Runs Batted In, 타점',
       'Stolen Base, 도루성공', 'Caught Stealing, 도루실패', 'Based on Balls, 4구',
       'Hit By Pitch, 사구', 'Intentional Based on Ball, 고의4구', 'Strike Out, 삼진',
       'Grounded into Double Play, 병살타', 'Sacrifice Hit, 희생타(희생번트)',
       'Sacrifice Fly, 희생플라이', 'Wins Above Replacement, 대체 선수 대비 승리 기여도',
       'Average, 타율', 'On Base Percentage, 출루율', 'Slugging Average, 장타율',
       'On base Plus Slugging, OPS, 출루율+장타율',
       'Runs per effective Plate Appearance, R/ePA',
       'Weighted Runs Created plus, 파크팩터 반영된 조정 득점 창출력', 'name', 'p_no',
       'p_pos'],
      dtype='object')

In [ ]:
pitching

,"Year, 연도","Team, 팀","Age, 나이(만)","Position, 포지션","Games, 출장","Games Started, 선발","Games Relief, 구원","Games Finished, 경기의 마지막 투수","Wins , 승","Loses, 패",...,"Wild Pitch, 폭투","Earned Run Average, 평균자책점","Runs Allowed 9th Innings, 9이닝 당 실점","responsibility Runs Allowed 9th Innings, 9이닝 당 책임실점","Fielding Independent Pitching, 수비 무관 평균자책점","Walks and Hits Per Inning Pitched, 이닝 당 출루허용률","Wins Above Replacement, 대체 선수 대비 승리 기여도",name,p_no,p_pos
0,2021,KIA,19,P,19,19,0,0,4,5,...,5,3.61,3.99,3.90,3.96,1.32,2.80,이의리,14788,pitching
1,2022,KIA,20,P,29,28,1,0,10,10,...,7,3.86,4.62,4.61,4.23,1.31,2.32,이의리,14788,pitching


In [12]:
len(list(pitching.columns))

37